In [1]:
"""
   This section of the code simply sets up all possible variables we might want to change during training.
"""
import os, sys, random, gzip, optparse
import numpy as np                     # Math and Deep Learning libraries
import torch                
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm                  # Pretty status bars
from collections import defaultdict
np.seterr(divide='ignore')             # Ignore divide by zero errors
np.warnings.filterwarnings('ignore')

# Use a GPU when possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

parser = optparse.OptionParser(description='FIND')
parser.add_option('--name',       type=str,            default='')
parser.add_option('--batch-size', type=int,            default=32,    help='batch size')
parser.add_option('--epochs',     type=int,            default=101,   help='number of epochs')
parser.add_option('--hidden-dim', type=int,            default=256,   help='hidden dim')
parser.add_option('--log',        type=str,            default='./logs/')
parser.add_option('--load',       type=str,            default='',    help='Load model')
parser.add_option('--load_test',  type=str,            default='',    help='Load specific test file')
parser.add_option('--binary',     type=str,            default='',    help='Binary classifer based on argument')
parser.add_option('--lr',         type=float,          default=5e-3)
parser.add_option('--nitro',      action='store_true', default=False, help='Train/Test on nitrogenase')
parser.add_option('--bd',         action='store_true', default=False, help='Train/Test on cytochrome bd oxidase')
parser.add_option('--test',       action='store_true', default=False, help='Swap validation for test set')
parser.add_option('--confusion',  action='store_true', default=False, help='If test, generate confusion matrix')
parser.add_option('--pconfusion', action='store_true', default=False, help='sum probabilities for confusion')
parser.add_option('--reweight',   action='store_true', default=False)


sys.argv = sys.argv[3:]  # Remove pykernel launcher
args, _ = parser.parse_args()

In [2]:
###############################################################################
##    Infrastructure to process Data into Numpy Arrays of Integers
##    Here we specify the dataset (indicated by args.nitro, args.bd or default HCO)
##    Then we convert letters to numbers via the acids dictionary.
##    We also compute the set of function labels (optionally training with binary)
###############################################################################
DATA_DIR = "data/"
acids = {' ':0, 'A':1, 'C':2, 'E':3, 'D':4, 'G':5, 'F':6, 'I':7, 'H':8, \
         'K':9, 'M':10, 'L':11, 'N':12, 'Q':13, 'P':14, 'S':15, 'R':16, \
         'T':17, 'W':18, 'V':19, 'Y':20, 'X':21 }
ints = {}
for v in acids:
    ints[acids[v]] = v

lbls = {}
ilbls = {}

if args.nitro:
    labels_file = "nitro.labels.txt"
elif args.bd:
    labels_file = "bd.labels.txt"
else:
    labels_file = "hco.labels.txt"

L = [line.strip() for line in open(DATA_DIR + labels_file,'r')]
# For logging we will store details of our training regime in the file name
run = "binary" if args.binary else "multi"
run += ".e{}".format(args.epochs)
run += ".h{}".format(args.hidden_dim)
run += ".n{}".format(args.name)
if args.reweight:
    run += ".reweight"

if len(args.binary) == 0:
    for v in L:
        lbls[v] = len(lbls)
        ilbls[lbls[v]] = v
    num_labels = len(lbls)
else:
    for v in L:
        if v == args.binary:
            lbls[v] = 1
            ilbls[lbls[v]] = v
        else:
            lbls[v] = 0
            ilbls[lbls[v]] = "OTHER"
    print(lbls, ilbls)
    num_labels = 2
    run += "." + args.binary

In [3]:
###############################################################################
##    This block introduces helper functions
##    to_int and to_string convert AAs back and forth between representations
##    sort_data and pad_data help create batches of data of a fixed length to pass
##      to the network.
###############################################################################

def to_int(seq):
    """   Map AA sequence to integers  """
    seq = seq.replace("*","")
    conv = []
    for i in range(len(seq)):
        if seq[i] not in acids:
            print(i, seq)
        conv.append(acids[seq[i]])
    return np.array(conv)

def to_string(seq):
    """  Map ints to AA sequence  """
    return "".join([ints[s] for s in seq])

def sort_data(inputs, outputs, strs=[]):
    """ 
      Sorted by input length and then output length
    """
    ignore_strs = False
    if len(strs) == 0:
        ignore_strs = True
        strs = [""]*len(inputs)
    v = []
    for i, o, s in zip(inputs, outputs, strs):
        v.append((len(i), i, o, s))
    v.sort(key=lambda x: x[0])

    sorted_inputs = []
    sorted_outputs = []
    sorted_strs= []
    for len_i, i, o, s in v:
        sorted_inputs.append(i)
        sorted_outputs.append(o)
        sorted_strs.append(s)

    if ignore_strs:
        return sorted_inputs, sorted_outputs
    else:
        return sorted_inputs, sorted_outputs, sorted_strs

def pad_data(inputs):
    max_i = max([len(i) for i in inputs])
  
    padded_i = np.zeros((len(inputs), max_i), dtype=np.int64)
    for i in range(len(inputs)):
        padded_i[i, :len(inputs[i])] = np.copy(inputs[i])

    return padded_i

In [4]:
###############################################################################
##    Data is loaded, Train/Validation/Test, counted, converted to numbers and 
##    stored in numpy arrays.
###############################################################################
""" Data & Parameters """
if args.nitro:
    prefix = "nitro.labeled" 
elif args.bd:
    prefix = "bd.labeled"
else:
    prefix = "hco.labeled"

data = [line.strip().split() for line in open(DATA_DIR + prefix + ".train",'r')]
if args.test:
    val = [line.strip().split() for line in open(DATA_DIR + prefix + ".test",'r')]
elif args.load_test != '':
    val = [line.strip().split() for line in open(args.load_test,'r')]
else:
    val = [line.strip().split() for line in open(DATA_DIR + prefix + ".val",'r')]

for vals in data:
    if len(vals) != 2:
        print(vals)
strs = np.array([sequence for label, sequence in data])
inputs = [to_int(sequence) for label, sequence in data]
outputs = np.array([lbls[label] for label, sequence in data])

print("Training counts\t")
l_c = defaultdict(int) 
for v in outputs:
    l_c[ilbls[v]] += 1
V = [(l_c[v],v) for v in l_c]
V.sort()
V.reverse()
print("    ".join(["{}: {}".format(lbl, cnt) for cnt,lbl in V]))

count = np.zeros(len(ilbls), dtype=np.float32)

for v in range(len(ilbls)):
    if ilbls[v] in l_c:
        count[v] += l_c[ilbls[v]]
distr = 1. - count/np.sum(count)
weight = torch.from_numpy(100*distr).to(device)

inps, outs, strs = sort_data(inputs, outputs, strs)
outs = np.array(outs)
strs = np.array(strs)

t_strs = np.array([sequence for label, sequence in val])
t_inps = [to_int(sequence) for label, sequence in val]
t_outs = np.array([lbls[label] for label, sequence in val])

print("Train Inps: ", len(inputs))
print("Train Outs: ", outputs.shape)
print("Test  Inps: ", len(t_inps))
print("Test  Outs: ", t_outs.shape)
print("Labels\t",lbls)

Training counts	
A: 3471    C: 794    B: 574    QNOR: 284    ENOR: 217    SNOR: 144    CNOR: 103    D: 66    GNOR: 44    E: 42    BNOR: 34    NNOR: 32    F: 31    H: 16    I: 14    J: 11    G: 11    K: 5
Train Inps:  5893
Train Outs:  (5893,)
Test  Inps:  812
Test  Outs:  (812,)
Labels	 {'QNOR': 0, 'H': 1, 'CNOR': 2, 'F': 3, 'GNOR': 4, 'I': 5, 'J': 6, 'K': 7, 'NNOR': 8, 'A': 9, 'BNOR': 10, 'ENOR': 11, 'C': 12, 'SNOR': 13, 'D': 14, 'B': 15, 'E': 16, 'G': 17}


In [5]:
###############################################################################
##    Model definition + Helper Functions
###############################################################################
class Net(nn.Module):
    def __init__(self, width=3, RF=19):
        """
           Build a stack of 1D convolutions with batch norm and ReLU activations
           The final two convolutions are simply linear layers, then followed by
           a prediction and attention layer.
        """
        super(Net, self).__init__()
        self.width = width
        self.RF = RF
    
        self.embedding = nn.Embedding(len(acids), args.hidden_dim)
        layers = [
          nn.Conv1d(args.hidden_dim, args.hidden_dim, self.width),
          nn.ReLU(),
          nn.BatchNorm1d(args.hidden_dim),
          nn.Conv1d(args.hidden_dim, args.hidden_dim, self.width*2),
          nn.ReLU(),
          nn.BatchNorm1d(args.hidden_dim),
          nn.Conv1d(args.hidden_dim, args.hidden_dim, self.width*4),
          nn.ReLU(),
          nn.BatchNorm1d(args.hidden_dim),
          nn.Conv1d(args.hidden_dim, args.hidden_dim, 1),
          nn.ReLU(),
          nn.Conv1d(args.hidden_dim, args.hidden_dim, 1),
          nn.ReLU(),
        ]

        self.conv_stack = nn.Sequential(*layers)
    
        self.pred = nn.Conv1d(args.hidden_dim, num_labels, 1)
        self.att = nn.Conv1d(args.hidden_dim, 1, 1)

    def forward(self, x):
        embed = self.embedding(x).permute(0,2,1)
        embed = self.conv_stack(embed)

        # Log probabilities for every class at every substring
        logits = self.pred(embed)
    
        # Un-normalized weight of a given n-gram
        att = self.att(embed)
        # Reshape [b,L] --> [b,1,L]  -- and normalize
        re_att = F.softmax(att.view(x.size()[0],1,-1), dim=-1)
        # Rescale logits by attention weight
        joint = re_att * logits
        # Class distribution
        collapsed = torch.sum(joint, 2)
        return collapsed, att, logits

    def reset_counts(self, epoch):
        self.gold_counts = np.zeros(num_labels)
        self.pred_counts = np.zeros(num_labels)
        self.corr_counts = np.zeros(num_labels)
        self.epoch = epoch

    """ Helper Functions """
    def run_evaluation(self, v_inputs, v_outputs, aggregate=False, verbose=False):
        net.train(mode=False)
        """
          Run evaluation
        """
        val_loss = 0.0
        val_acc = []
        gold_counts = np.zeros(num_labels)
        pred_counts = np.zeros(num_labels)
        corr_counts = np.zeros(num_labels)

        if args.confusion:
            pairs = np.zeros((num_labels, num_labels))

        v_inps, v_outs = sort_data(v_inputs, v_outputs)
        v_outs = np.array(v_outs)
        batches = []
        indices = list(range(len(v_inps)))
        for start in range(0, len(indices), args.batch_size):
            batches.append((start, min(args.batch_size, len(indices)-start)))

        for start, b_size in tqdm(batches, ncols=80):
            vals = indices[start : start + b_size]
      
            inputs = torch.from_numpy(pad_data(v_inps[indices[start]:indices[start+b_size-1]+1])).to(device)
            labels = torch.from_numpy(v_outs[vals]).to(device)
            logits, att, full = net(inputs)
            att = F.softmax(torch.squeeze(att), dim=-1)
            val_loss += F.cross_entropy(logits, labels).item()
            _, preds = torch.max(logits, 1)
      
            preds = preds.data.cpu().numpy()
            val_acc.extend(list((preds == v_outs[vals])))
      
            np.add.at(pred_counts, preds, 1)
            np.add.at(gold_counts, v_outs[vals], 1)
            np.add.at(corr_counts, preds[(preds == v_outs[vals])], 1)

            if args.confusion:
                if not args.pconfusion:
                    np.add.at(pairs, [v_outs[vals], preds], 1)
                else:
                    dists = F.softmax(logits, -1)
                    for i in range(len(vals)):
                        gold = v_outs[vals][i]

                        tmp = [(dists[i,j], j) for j in range(len(ilbls))]
                        tmp.sort()
                        prob, second = tmp[-2]
                        pairs[gold, second] += 1

            if aggregate:
                self.aggregate_predictors(t_strs[vals], v_outs[vals], full, att)

        if verbose:
            if aggregate:
                self.print_eval((gold_counts, pred_counts, corr_counts))
            else:
                self.print_eval((self.gold_counts, self.pred_counts, self.corr_counts), 
                               (gold_counts, pred_counts, corr_counts))

        if args.confusion:
            out = open("confusion.csv", 'w')
            out.write("," + ",".join([ilbls[i] for i in range(len(ilbls))]) + "\n")
            for i in range(len(ilbls)):
                out.write("{},".format(ilbls[i]))
                for j in range(len(ilbls)):
                    out.write("{},".format(pairs[i,j]))
                out.write("\n")
            out.close()
        return val_loss, 100*np.array(val_acc).mean()

    def aggregate_predictors(self, seqs, outs, full, att):
        dists = F.softmax(full.permute(0, 2, 1), dim=-1)
        if dists.shape[0] == 1:
            att = att.unsqueeze(0)  # batch size of 1 needs to be unsqueezed
        vals = dists * att.unsqueeze(2)
        for b in range(len(seqs)):
            max_val = -1e10
            max_predictor = "NONE"
            max_class = -1
            for i in range(len(att[0])):
                predictor = seqs[b][i:i + self.RF]

            for c in range(num_labels):
                self.predictors[(predictor, c)] += vals[b,i,c].item() 

            cval, c = torch.max(vals[b,i,:], 0)
            cval = cval.item()
            if cval > max_val:
                max_val = cval
                max_predictor = predictor
                max_class = c.item()
            self.top_predictor.append((max_val, max_predictor, ilbls[max_class], 
                                       ilbls[outs[b]], seqs[b].strip()))


    def print_predictors(self, epoch):
        g = gzip.open("{}.predictors.{}.{}.joint.gz".format(run, self.RF, epoch),'wt')
        joint = defaultdict(list)
        for seq, lbl in self.predictors:
            joint[ilbls[lbl]].append((self.predictors[(seq, lbl)], seq))
    
        for lbl in joint:
            vals = joint[lbl]
            vals.sort()
            vals.reverse()
            for val, seq in vals:
                g.write("{:5} {:30} {}\n".format(lbl, seq, val))
            g.write("\n")
        g.close()
   
        start = "nitro" if args.nitro else "bd" if args.bd else "hco"
        rtype = "binary" if args.binary else "multi"
        rewht = "reweight" if args.reweight else "orig"
        g = gzip.open("{}.{}.{}.{}.{}.h{}.{}.top_predictors.txt.gz".format(start, epoch, 
                  self.RF, rtype, rewht, args.hidden_dim, args.name), 'wt')
        g.write("{:10} {:30} {:5} {:5} {}\n".format("Val", "Predictor", "Pred", "Gold", "Seq"))
        self.top_predictor.sort()
        self.top_predictor.reverse()
        for val, predictor, pred, gold, seq in self.top_predictor:
            g.write("{:10.9f} {:30} {:5} {:5} {}\n".format(val, predictor, pred, gold, seq))
        g.close()


    def print_eval(self, train, test=None): 
        """  Print training performance  """
        gold, pred, corr = train
        p, r = corr / pred, corr / gold
        f = 2*p*r/(p+r)
    
        if test is not None:
            t_gold, t_pred, t_corr = test
            t_p, t_r = t_corr / t_pred, t_corr / t_gold
            t_f = 2*t_p*t_r/(t_p+t_r)
    
        gold_counts = [(gold[lab],lab) for lab in range(num_labels)]
        gold_counts.sort(reverse=True)
        for count, i in gold_counts:
            train_str = "{:<10} {:<5} {:5.3f} {:5.3f} {:5.3f}   ".format(ilbls[i], int(count), p[i], r[i], f[i])
            if test is not None:
                test_str = "{:<5} {:5.3f} {:5.3f} {:5.3f}".format(int(t_gold[i]), t_p[i], t_r[i], t_f[i])
            else:
                test_str = ""
            print(train_str + test_str)

In [6]:
###############################################################################
##    Training Loop
###############################################################################
#torch.cuda.manual_seed(20180119)  <-- set a value for consistency
if args.load != '':
    net = torch.load(args.load)                             # Load Saved Model
    net.to(device)
    if args.confusion:
        loss, acc = net.run_evaluation(t_inps, t_outs, aggregate=args.load_test != '')
    else:
        loss, acc = net.run_evaluation(t_inps, t_outs, aggregate=args.load_test != '')
    print("Acc: {:5.3f}".format(acc))
    if args.load_test != '':
        net.print_predictors("test")
    sys.exit()
else:
    net = Net()
    net.to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=args.lr)

"""
  Perform training
"""
prev_acc = 0
for epoch in range(0, args.epochs):
    net.predictors = defaultdict(int)
    net.top_predictor = []

    batches = []
    indices = list(range(len(inps)))
    for start in range(0, len(indices), args.batch_size):
        batches.append((start, min(args.batch_size, len(indices)-start)))
    random.shuffle(batches)

    total_loss = 0.0
    train_acc = []
  
    net.reset_counts(epoch)
    for start, b_size in tqdm(batches, ncols=80):
        r = indices[start : start + b_size]
        
        # Setup
        optimizer.zero_grad()
        inputs = torch.from_numpy(pad_data(inps[indices[start] : indices[start+b_size-1] + 1])).to(device)
        labels = torch.from_numpy(outs[r]).to(device)

        # Predict
        net.train(mode=True)
        logits, att, full = net(inputs)
        if args.reweight:
            ce_loss = F.cross_entropy(logits, labels, weight=weight)
        else:
            ce_loss = F.cross_entropy(logits, labels)
        att = F.softmax(torch.squeeze(att), dim=-1)
    
        # Compute loss and update
        loss = ce_loss
        total_loss += ce_loss.item()
        loss.backward()
        optimizer.step()
    
        # Look at predictions
        _, preds = torch.max(logits, 1)
        dists = full.permute(0,2,1).cpu().data.numpy()
    
        preds = preds.data.cpu().numpy()
        np.add.at(net.pred_counts, preds, 1)
        np.add.at(net.gold_counts, outs[r], 1)
        np.add.at(net.corr_counts, preds[preds == outs[r]], 1)

        train_acc.extend(list(preds == outs[r]))
    
        # Very slow and memory intensive, should only be done with final model
        # Useful for performing actual analysis.
        if epoch == args.epochs - 1: 
            net.aggregate_predictors(strs[r], outs[r], full, att)
  
    # Evaluate on validation (during training)
    val_loss, val_acc = net.run_evaluation(t_inps, t_outs, verbose=(epoch % 5 == 0 or epoch == args.epochs -1))

    print("Epoch: {}  Train Loss: {:8.4f}  Acc: {:5.2f}  Val  Loss {:8.4f}  Acc: {:5.2f}".format(epoch, 
          total_loss, 100*np.array(train_acc).mean(), val_loss, val_acc))

    # Save best validation model for optimal generalization
    if val_acc > prev_acc:
        prev_acc = val_acc
        pref = "nitro" if args.nitro else "bd" if args.bd else "hco"
        torch.save(net, "{}.{}.model".format(pref, run))

out = torch.cat((net.embedding.weight.data, torch.ones(len(acids), 1).to(device)), 1)

# Generate analysis
net.print_predictors("final")

  1%|▍                                          | 2/185 [00:00<00:09, 18.77it/s]

A          3471  0.589 0.994 0.740   499   0.615 1.000 0.761
C          794     nan 0.000   nan   115     nan 0.000   nan
B          574   0.000 0.000   nan   71      nan 0.000   nan
QNOR       284     nan 0.000   nan   32      nan 0.000   nan
ENOR       217     nan 0.000   nan   29      nan 0.000   nan
SNOR       144     nan 0.000   nan   23      nan 0.000   nan
CNOR       103     nan 0.000   nan   10      nan 0.000   nan
D          66      nan 0.000   nan   6       nan 0.000   nan
GNOR       44      nan 0.000   nan   6       nan 0.000   nan
E          42      nan 0.000   nan   6       nan 0.000   nan
BNOR       34      nan 0.000   nan   4       nan 0.000   nan
NNOR       32      nan 0.000   nan   3       nan 0.000   nan
F          31      nan 0.000   nan   3       nan 0.000   nan
H          16      nan 0.000   nan   1       nan 0.000   nan
I          14      nan 0.000   nan   1       nan 0.000   nan
G          11      nan 0.000   nan   1       nan 0.000   nan
J          11    0.000 0

  2%|▋                                          | 3/185 [00:00<00:08, 22.67it/s]

Epoch: 1  Train Loss: 192.6457  Acc: 67.11  Val  Loss  24.0804  Acc: 69.95


  1%|▍                                          | 2/185 [00:00<00:10, 17.76it/s]

...

Epoch: 98  Train Loss:  25.0486  Acc: 95.37  Val  Loss   3.7334  Acc: 96.67


  1%|▏                                          | 1/185 [00:00<00:20,  8.98it/s]

Epoch: 99  Train Loss:  33.4907  Acc: 95.60  Val  Loss   3.3804  Acc: 96.67


100%|███████████████████████████████████████████| 26/26 [00:00<00:00, 78.12it/s]


A          3471  1.000 0.992 0.996   499   1.000 0.988 0.994
C          794   1.000 0.999 0.999   115   1.000 1.000 1.000
B          574   0.977 0.981 0.979   71    0.971 0.958 0.965
QNOR       284   0.979 0.982 0.981   32    0.970 1.000 0.985
ENOR       217   0.876 0.977 0.924   29    1.000 1.000 1.000
SNOR       144   0.821 0.986 0.896   23    0.697 1.000 0.821
CNOR       103   1.000 0.981 0.990   10    1.000 1.000 1.000
D          66    0.754 0.652 0.699   6     1.000 0.500 0.667
GNOR       44    1.000 0.955 0.977   6     1.000 1.000 1.000
E          42    0.737 1.000 0.848   6     1.000 1.000 1.000
BNOR       34    0.756 1.000 0.861   4     0.571 1.000 0.727
NNOR       32    1.000 0.125 0.222   3     0.000 0.000   nan
F          31    0.886 1.000 0.939   3     1.000 1.000 1.000
H          16    0.321 0.562 0.409   1     0.500 1.000 0.667
I          14    0.000 0.000   nan   1       nan 0.000   nan
G          11    0.364 0.364 0.364   1     1.000 1.000 1.000
J          11      nan 0

In [7]:
# Above is an example run of 100 epochs with training (left) and validation (right) 
# precision, recall and F1 printed for every class.  Additionally, we print the overall 
# losses and accuracies after every epoch.  Note, these do not decrease monitonically but
# in general the model does perform better after some backtracking.  For most use cases a 
# shorter training regime is probably sufficient.